In [1]:
import os

import gmsh
import matplotlib.pyplot as plt
import meshio
import numpy as np
import pandas as pd

import commons, geometry, utils

In [3]:
Lsep = 50
Lx = 470
Ly = 470
Lcat = 60
Rp = 6
eps_se = 0.5
eps_am = 1 - eps_se
markers = commons.SurfaceMarkers()
phases = commons.Phases()
CELL_TYPES = commons.CellTypes()
start_zpos = 50
scale = (800/470) * 1e-6
scale_factor = [scale, scale, scale]
outdir = "mesh/laminate/"
utils.make_dir_if_missing(outdir)
msh_path = os.path.join(outdir, 'laminate.msh')
tetr_path = os.path.join(outdir, 'tetr.xdmf')
tria_path = os.path.join(outdir, 'tria.xdmf')

## Build Laminate Geometry of Cylindrical SE Strands

In [4]:
df = 470 * pd.read_csv('data/laminate.csv')

In [13]:
gmsh.initialize()
gmsh.model.add('laminate')
gmsh.option.setNumber('General.Verbosity', 1)
box_se = gmsh.model.occ.addBox(0, 0, 0, Lx, Ly, Lsep + Lcat)
gmsh.model.occ.synchronize()
box_am = gmsh.model.occ.addBox(0, 0, Lsep + Lcat, Lx, Ly, Rp)
gmsh.model.occ.synchronize()
cylinders = []
spheres = []

for idx in range(df.shape[0]):
    x, y, _ = df.loc[idx, :]
    if (x + Rp >= 470) or (y + Rp >= 470):
        continue
    if (x - Rp <= 0) or (y - Rp <= 0):
        continue
    cyl = gmsh.model.occ.addCylinder(x, y, Lsep, 0, 0, Lcat, Rp)
    # cylinders.append(cyl)
    gmsh.model.occ.synchronize()
    sphere = gmsh.model.occ.addSphere(x, y, Lsep, Rp)
    rod = gmsh.model.occ.fuse([(3, sphere)], [(3, cyl)], removeTool=True, removeObject=True)
    print(rod)
    cylinders.append(rod[0][0][1])
    gmsh.model.occ.synchronize()

se_phase = gmsh.model.occ.cut([(3, box_se)], [(3, c) for c in cylinders], removeTool=False)
gmsh.model.occ.synchronize()
union = gmsh.model.occ.fuse([(3, box_am)], [(3, c) for c in cylinders])
gmsh.model.occ.synchronize()

vols = gmsh.model.occ.getEntities(3)
se_vol = gmsh.model.addPhysicalGroup(3, [vols[0][1]], phases.electrolyte)
am_vol = gmsh.model.addPhysicalGroup(3, [vols[1][1]], phases.active_material)
gmsh.model.occ.synchronize()

right = []
left = []
insulated = []
interface = []
for surf in gmsh.model.occ.getEntities(2):
    com = gmsh.model.occ.getCenterOfMass(surf[0], surf[1])
    if np.isclose(com[2], Lsep + Lcat + Rp):
        right.append(surf[1])
    elif np.isclose(com[2], Lsep + Lcat + 0.5 * Rp):
        if np.isclose(com[1], 235) or np.isclose(com[0], 235):
            insulated.append(surf[1])
        else:
            interface.append(surf[1])
    elif np.isclose(com[2], 0.5 * (Lsep + Lcat)):
        if np.isclose(com[1], 235) or np.isclose(com[0], 235):
            insulated.append(surf[1])
        else:
            interface.append(surf[1])
    elif np.isclose(com[2], 0):
        left.append(surf[1])
    else:
        interface.append(surf[1])

gmsh.model.addPhysicalGroup(2, left, markers.left_cc)
gmsh.model.addPhysicalGroup(2, right, markers.right_cc)
gmsh.model.addPhysicalGroup(2, insulated, markers.insulated)
gmsh.model.addPhysicalGroup(2, interface, markers.am_se_interface)
gmsh.model.occ.synchronize()
gmsh.model.mesh.generate(3)
gmsh.write(msh_path)
gmsh.finalize()

([(3, 3)], [[], []])
([(3, 4)], [[], []])
([(3, 5)], [[], []])
([(3, 6)], [[], []])
([(3, 7)], [[], []])
([(3, 8)], [[], []])
([(3, 9)], [[], []])
([(3, 10)], [[], []])
([(3, 11)], [[], []])
([(3, 12)], [[], []])
([(3, 13)], [[], []])
([(3, 14)], [[], []])
([(3, 15)], [[], []])
([(3, 16)], [[], []])
([(3, 17)], [[], []])
([(3, 18)], [[], []])
([(3, 19)], [[], []])
([(3, 20)], [[], []])
([(3, 21)], [[], []])
([(3, 22)], [[], []])
([(3, 23)], [[], []])
([(3, 24)], [[], []])
([(3, 25)], [[], []])
([(3, 26)], [[], []])
([(3, 27)], [[], []])
([(3, 28)], [[], []])
([(3, 29)], [[], []])
([(3, 30)], [[], []])
([(3, 31)], [[], []])
([(3, 32)], [[], []])
([(3, 33)], [[], []])
([(3, 34)], [[], []])
([(3, 35)], [[], []])
([(3, 36)], [[], []])
([(3, 37)], [[], []])
([(3, 38)], [[], []])
([(3, 39)], [[], []])
([(3, 40)], [[], []])
([(3, 41)], [[], []])
([(3, 42)], [[], []])
([(3, 43)], [[], []])
([(3, 44)], [[], []])
([(3, 45)], [[], []])
([(3, 46)], [[], []])
([(3, 47)], [[], []])
([(3, 48)], [[], 

In [15]:
msh = meshio.read(msh_path)
tetr_unscaled = geometry.create_mesh(msh, CELL_TYPES.tetra)
tetr_unscaled.write(tetr_path)
tetr_scaled = geometry.scale_mesh(tetr_unscaled, CELL_TYPES.tetra, scale_factor=scale_factor)
tetr_scaled.write(tetr_path)
tria_unscaled = geometry.create_mesh(msh, CELL_TYPES.triangle)
tria_unscaled.write(tria_path)
tria_scaled = geometry.scale_mesh(tria_unscaled, CELL_TYPES.triangle, scale_factor=scale_factor)
tria_scaled.write(tria_path)